<a href="https://colab.research.google.com/github/tatendatobaiwa/cnn/blob/main/CNN_Medical_Imaging_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***SIMPLE CNN MODEL BUILD FOR LUNG CANCER IMAGING***


**Step 1: Mount Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Step 2: Create Directories in Drive**

**Step 3: Install Dependenices**

In [2]:
!pip install -q tensorflow==2.13.0
!pip install -q tensorflow-addons==0.23.0
!pip install albumentations==1.3.0
!pip install numpy==1.24.3

In [3]:
import tensorflow as tf
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import os
import sys
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

project_path = '/content/drive/MyDrive/CNN_Medical_Imaging_Project'
sys.path.append(project_path)

**Mini Step 3a: Verify Fixes**

In [4]:
import tensorflow as tf
import tensorflow_addons as tfa

print(f"TensorFlow version: {tf.__version__}")
print(f"TensorFlow Addons version: {tfa.__version__}")

TensorFlow version: 2.13.0
TensorFlow Addons version: 0.23.0


/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


**Step 4: data_processing.py: Load and Preprocess Data**

In [5]:
import cv2
import numpy as np
from pathlib import Path
import albumentations as A
from tensorflow.keras.utils import Sequence

class AlbumentationsSequence(Sequence):
    """Custom sequence for batching and augmenting images."""
    def __init__(self, images, labels, batch_size, transform):
        self.images = images  # Array of images (uint8)
        self.labels = labels  # Array of integer labels
        self.batch_size = batch_size
        self.transform = transform

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = min((idx + 1) * self.batch_size, len(self.images))
        batch_images = self.images[start:end]
        batch_labels = self.labels[start:end]
        # Apply augmentations and convert to 3-channel RGB
        augmented = [self.transform(image=img)['image'] for img in batch_images]
        augmented = np.array(augmented)
        augmented = np.stack((augmented,) * 3, axis=-1)  # Grayscale to RGB
        return augmented, batch_labels

class DataProcessor:
    """Handles loading and preprocessing of image data."""
    def __init__(self, target_size=(224, 224)):  # Fixed typo in target_size
        self.target_size = target_size
        self.class_names = ['benign', 'malignant', 'normal']

    def load_images(self, data_dir):
        """Load images and labels from a directory with class subfolders."""
        data_dir = Path(data_dir)
        images = []
        labels = []
        print(f"Loading images from {data_dir}")
        for label, class_name in enumerate(self.class_names):
            class_dir = data_dir / class_name
            if not class_dir.exists():
                print(f"Warning: Directory {class_dir} does not exist.")
                continue
            img_paths = list(class_dir.glob('*.[jp][pn][gf]'))  # Match .jpg, .jpeg, .png
            print(f"Found {len(img_paths)} images in {class_name}")
            for img_path in img_paths:
                img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"Warning: Failed to load {img_path}")
                    continue
                img = cv2.resize(img, self.target_size)
                images.append(img)
                labels.append(label)
        if not images:
            print("No images loaded.")
        else:
            print(f"Total images loaded: {len(images)}")
        return np.array(images), np.array(labels, dtype=np.int32), self.class_names

    def create_generators(self, X_train, y_train, X_val, y_val, batch_size=32):
        """Create training and validation generators with augmentation."""
        train_transform = A.Compose([
            A.Rotate(limit=20, p=0.5),
            A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=0, p=0.5),
            A.HorizontalFlip(p=0.5),
            A.ElasticTransform(alpha=34, sigma=4, p=0.3),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
            A.Normalize(mean=0.0, std=1.0),
        ])
        val_transform = A.Compose([
            A.Normalize(mean=0.0, std=1.0),
        ])
        train_gen = AlbumentationsSequence(X_train, y_train, batch_size, train_transform)
        val_gen = AlbumentationsSequence(X_val, y_val, batch_size, val_transform)
        return train_gen, val_gen

    def load_test_images(self, test_dir):
        """Load and preprocess test images."""
        return self.load_images(test_dir)  # Reuses load_images for consistency

if __name__ == "__main__":
    # Test usage
    processor = DataProcessor()
    X_train, y_train, class_names = processor.load_images('/content/drive/MyDrive/CNN_Medical_Imaging_Project/data/raw/train')
    print(f"Loaded {len(X_train)} training images")

Loading images from /content/drive/MyDrive/CNN_Medical_Imaging_Project/data/raw/train
Found 120 images in benign
Found 335 images in malignant
Found 416 images in normal
Total images loaded: 871
Loaded 871 training images


**Step 5: train.py: Model Training**

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import numpy as np
from data_processing import DataProcessor
from model_mobilenet import create_mobilenet_model

def train_model(train_gen, val_gen, class_names, epochs=50):
    # Ensure train_gen.labels is a NumPy array of integers
    if not isinstance(train_gen.labels, np.ndarray):
        train_gen.labels = np.array(train_gen.labels)
    train_gen.labels = train_gen.labels.astype(int)

    # Debugging prints
    print("Type of train_gen.labels:", type(train_gen.labels))
    print("Shape of train_gen.labels:", train_gen.labels.shape)
    print("First few values of train_gen.labels:", train_gen.labels[:5] if len(train_gen.labels) > 0 else "Empty")

    # Validate train_gen.labels
    if len(train_gen.labels) == 0:
        raise ValueError("train_gen.labels is empty. No training labels available.")

    # Compute unique classes
    classes = np.unique(train_gen.labels)
    print("Classes:", classes)
    if len(classes) == 0:
        raise ValueError("No unique classes found in train_gen.labels.")

    print("Type of classes[0]:", type(classes[0]))

    # Compute class weights
    class_weights = compute_class_weight('balanced', classes=classes, y=train_gen.labels)
    class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

    # Create and compile the model
    model = create_mobilenet_model(num_classes=len(class_names))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Define callbacks
    early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

    # Initial training
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        callbacks=[early_stop, checkpoint, reduce_lr],
        class_weight=class_weights_dict
    )

    # Fine-tuning
    set_trainable = False
    for layer in model.layers:
        if layer.name == 'block_13_expand':
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False

    # Recompile for fine-tuning
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Fine-tuning training
    history_fine = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        callbacks=[early_stop, checkpoint, reduce_lr],
        class_weight=class_weights_dict
    )

    return model, history, history_fine

# Main execution
if __name__ == "__main__":
    processor = DataProcessor()

    # Load all data from the training directory
    X_train_full, y_train_full, class_names = processor.load_images('/content/drive/MyDrive/CNN_Medical_Imaging_Project/data/raw/train')

    # Debugging loaded data
    print("X_train_full shape:", X_train_full.shape)
    print("y_train_full shape:", y_train_full.shape)
    print("First few y_train_full labels:", y_train_full[:5])
    print("Class names:", class_names)

    # Check if data was loaded
    if len(X_train_full) == 0 or len(y_train_full) == 0:
        raise ValueError("No training data loaded. Verify the training directory.")

    # Split the training data into 80% training and 20% validation
    X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, stratify=y_train_full, random_state=42)

    # Debugging split data
    print("X_train shape:", X_train.shape)
    print("X_val shape:", X_val.shape)
    print("y_train shape:", y_train.shape)
    print("y_val shape:", y_val.shape)

    # Create generators
    train_gen, val_gen = processor.create_generators(X_train, y_train, X_val, y_val)

    # Train the model
    model, history, history_fine = train_model(train_gen, val_gen, class_names)

    # Load test data for final evaluation
    X_test, y_test, _ = processor.load_images('/content/drive/MyDrive/CNN_Medical_Imaging_Project/data/raw/test')
    print("X_test shape:", X_test.shape)
    print("y_test shape:", y_test.shape)

    # Evaluate the model on the test set
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"Test loss: {test_loss:.4f}")
    print(f"Test accuracy: {test_acc:.4f}")

Loading images from /content/drive/MyDrive/CNN_Medical_Imaging_Project/data/raw/train
Found 120 images in benign
Found 335 images in malignant
Found 416 images in normal
Total images loaded: 871
X_train_full shape: (871, 224, 224)
y_train_full shape: (871,)
First few y_train_full labels: [0 0 0 0 0]
Class names: ['benign', 'malignant', 'normal']
X_train shape: (696, 224, 224)
X_val shape: (175, 224, 224)
y_train shape: (696,)
y_val shape: (175,)
Type of train_gen.labels: <class 'numpy.ndarray'>
Shape of train_gen.labels: (696,)
First few values of train_gen.labels: [0 1 1 1 2]
Classes: [0 1 2]
Type of classes[0]: <class 'numpy.int64'>
9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/50
22/22 [==============================] - 32s 1s/step - loss: 1.2499 - accuracy: 0.4885 - val_loss: 0.7865 - val_accuracy: 0.5086 - lr: 0.0010


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/50
22/22 [==============================] - 29s 1s/step - loss: 0.9135 - accuracy: 0.5718 - val_loss: 0.4980 - val_accuracy: 0.8114 - lr: 0.0010
Epoch 3/50
22/22 [==============================] - 34s 2s/step - loss: 0.7531 - accuracy: 0.6566 - val_loss: 0.5918 - val_accuracy: 0.6686 - lr: 0.0010
Epoch 4/50
22/22 [==============================] - 36s 2s/step - loss: 0.6906 - accuracy: 0.6739 - val_loss: 0.4143 - val_accuracy: 0.8286 - lr: 0.0010
Epoch 5/50
12/22 [===============>..............] - ETA: 10s - loss: 0.7186 - accuracy: 0.6463

**Step 6: evaluate.py: Model Evaluation**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import label_binarize

def evaluate_model(model, X_test, y_test, class_names, save_path=None):
    """
    Evaluate a trained model on test data with classification metrics and visualizations.

    Parameters:
    - model: Trained Keras model.
    - X_test: Test images (numpy array).
    - y_test: True labels (numpy array).
    - class_names: List of class names (e.g., ['benign', 'malignant', 'normal']).
    - save_path: Optional path to save confusion matrix plot (e.g., 'confusion_matrix.png').
    """
    # Predict probabilities and classes
    y_pred_probs = model.predict(X_test, verbose=0)
    y_pred_classes = np.argmax(y_pred_probs, axis=1)

    # Classification Report
    print("Classification Report:")
    report = classification_report(y_test, y_pred_classes, target_names=class_names)
    print(report)

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    if save_path:
        plt.savefig(save_path)
        print(f"Confusion matrix saved to {save_path}")
    plt.show()

    # Per-Class Sensitivity and Specificity
    print("\nPer-Class Metrics:")
    for i, class_name in enumerate(class_names):
        tp = cm[i, i]
        fn = np.sum(cm[i, :]) - tp
        fp = np.sum(cm[:, i]) - tp
        tn = np.sum(cm) - tp - fn - fp
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        print(f"{class_name}: Sensitivity = {sensitivity:.2f}, Specificity = {specificity:.2f}")

    # ROC-AUC Score
    y_test_bin = label_binarize(y_test, classes=range(len(class_names)))
    auc = roc_auc_score(y_test_bin, y_pred_probs, multi_class='ovr')
    print(f"\nROC-AUC Score (One-vs-Rest): {auc:.4f}")

if __name__ == "__main__":
    from tensorflow.keras.models import load_model
    from data_processing import DataProcessor

    # Load test data instead of validation data
    processor = DataProcessor()
    X_test, y_test, class_names = processor.load_test_images('/content/drive/MyDrive/CNN_Medical_Imaging_Project/data/raw/test')

    # Load the trained model
    model = load_model('best_model.h5')

    # Evaluate the model
    evaluate_model(model, X_test, y_test, class_names, save_path='confusion_matrix.png')

ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

**Step 7: xai.py: Implementation of Explainable AI (Grad-CAM)**

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

def get_gradcam_heatmap(model, img_array, layer_name):
    """
    Generate a Grad-CAM heatmap for a given image.

    Args:
        model: Trained Keras model.
        img_array: Input image array with shape (1, H, W, C).
        layer_name: Name of the convolutional layer to use for Grad-CAM.

    Returns:
        Heatmap as a numpy array.
    """
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, np.argmax(predictions[0])]  # Target the predicted class
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def superimpose_heatmap(img, heatmap, alpha=0.4):
    """
    Superimpose a Grad-CAM heatmap on the original image.

    Args:
        img: Original image (numpy array, shape (H, W, C), values in [0, 1]).
        heatmap: Grad-CAM heatmap (numpy array).
        alpha: Transparency factor for the heatmap overlay.

    Returns:
        Superimposed image as a numpy array (uint8).
    """
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * alpha + img * 255  # Scale img back to 0-255
    superimposed_img = np.clip(superimposed_img, 0, 255).astype(np.uint8)
    return superimposed_img

def visualize_explanations(model, X_test, y_test, class_names, layer_name, num_images=5, save_dir=None):
    """
    Visualize Grad-CAM explanations for test images.

    Args:
        model: Trained Keras model.
        X_test: Test images (numpy array, shape (N, H, W, C)).
        y_test: True labels (numpy array, shape (N,)).
        class_names: List of class names (e.g., ['benign', 'malignant', 'normal']).
        layer_name: Name of the layer to use for Grad-CAM.
        num_images: Number of images to visualize.
        save_dir: Optional directory to save the visualizations.
    """
    if save_dir and not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for i in range(min(num_images, len(X_test))):
        img = X_test[i:i+1]  # Batch of 1
        true_label = class_names[y_test[i]]
        heatmap = get_gradcam_heatmap(model, img, layer_name)
        superimposed_img = superimpose_heatmap(img[0], heatmap)

        # Predict class for the image
        pred_probs = model.predict(img, verbose=0)
        pred_class = class_names[np.argmax(pred_probs)]

        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(img[0][:, :, 0], cmap='gray')  # Show first channel (grayscale)
        plt.title(f'True: {true_label}\nPred: {pred_class}')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(superimposed_img)
        plt.title('Grad-CAM')
        plt.axis('off')

        if save_dir:
            plt.savefig(os.path.join(save_dir, f'gradcam_{i}_true_{true_label}_pred_{pred_class}.png'))
            print(f"Saved Grad-CAM for image {i} to {save_dir}")
        plt.show()

if __name__ == "__main__":
    from tensorflow.keras.models import load_model
    from data_processing import DataProcessor

    # Load test data
    processor = DataProcessor()
    X_test, y_test, class_names = processor.load_test_images('/content/drive/MyDrive/CNN_Medical_Imaging_Project/data/raw/test')

    # Load the trained model
    model = load_model('best_model.h5')  # Match the path from train.py

    # Visualize explanations
    layer_name = 'block_16_project'  # Adjusted for MobileNetV2, late conv layer
    visualize_explanations(model, X_test, y_test, class_names, layer_name, num_images=5, save_dir='gradcam_outputs')